In [1]:
from nltk.corpus import wordnet

In [2]:
def get_synset(word):
    return wordnet.synsets(word)[0]

def get_synset_list(words):
    synsets = []
    for word in words:
        synsets += [get_synset(word)]
    return synsets

## 1. Word glosses

In [3]:
def get_gloss(synset):
    print(f'{synset.name()}: {synset.definition()}')

def get_glosses(word):
    print(f'Glosses for the word "{word}":')

    for synset in wordnet.synsets(word):
        get_gloss(synset)

get_glosses('school')

Glosses for the word "school":
school.n.01: an educational institution
school.n.02: a building where young people receive education
school.n.03: the process of being formally educated at a school
school.n.04: a body of creative artists or writers or thinkers linked by a similar style or by similar teachers
school.n.05: the period of instruction in a school; the time period when school is in session
school.n.06: an educational institution's faculty and students
school.n.07: a large group of fish
school.v.01: educate in or as if in a school
educate.v.03: teach or refine to be discriminative in taste or judgment
school.v.03: swim in or form a large group of fish


## 2. Synonyms

In [4]:
def check_synonyms(word1, word2):
    def find_similarity(word1, word2):
        for synset in wordnet.synsets(word1):
            if synset.lemmas()[0].name() == word2:
                print(f'{word1, word2}: {synset.definition()}')
                return True
        return False

    if not find_similarity(word1, word2):
        find_similarity(word2, word1)

check_synonyms('force', 'strength')

('strength', 'force'): physical energy or intensity


## 3. Holonyms and meronyms

In [5]:
def holonyms_meronyms(synset):
    holonyms = synset.substance_holonyms() + synset.part_holonyms() + synset.member_holonyms()
    meronyms = synset.substance_meronyms() + synset.part_meronyms() + synset.member_meronyms()
    return (holonyms, meronyms)

synset = get_synset('tree')

print(f'Holonyms:\n' + \
      f'Substance: {synset.substance_holonyms()}\n' + \
      f'Part: {synset.part_holonyms()}\n' + \
      f'Member: {synset.member_holonyms()}\n')

print(f'Meronyms:\n' + \
      f'Substance: {synset.substance_meronyms()}\n' + \
      f'Part: {synset.part_meronyms()}\n' + \
      f'Member: {synset.member_meronyms()}\n')

print(f'Holonyms and meronyms for the synset "{synset.name()}":\n{holonyms_meronyms(synset)}')

Holonyms:
Substance: []
Part: []
Member: [Synset('forest.n.01')]

Meronyms:
Substance: [Synset('heartwood.n.01'), Synset('sapwood.n.01')]
Part: [Synset('burl.n.02'), Synset('crown.n.07'), Synset('limb.n.02'), Synset('stump.n.01'), Synset('trunk.n.01')]
Member: []

Holonyms and meronyms for the synset "tree.n.01":
([Synset('forest.n.01')], [Synset('heartwood.n.01'), Synset('sapwood.n.01'), Synset('burl.n.02'), Synset('crown.n.07'), Synset('limb.n.02'), Synset('stump.n.01'), Synset('trunk.n.01')])


## 4. Path of hypernyms

In [6]:
def hypernyms_path(synset):
    hypernyms = []

    while True:
        hypernyms += [synset]
        synset = synset.hypernyms()

        if not synset:
            break
        synset = synset[0]

    return hypernyms

synset = get_synset('tree')
print(hypernyms_path(synset))

[Synset('tree.n.01'), Synset('woody_plant.n.01'), Synset('vascular_plant.n.01'), Synset('plant.n.02'), Synset('organism.n.01'), Synset('living_thing.n.01'), Synset('whole.n.02'), Synset('object.n.01'), Synset('physical_entity.n.01'), Synset('entity.n.01')]


## 5. Closest common hypernyms

In [7]:
def closest_hypernyms(synset1, synset2):
    path1 = hypernyms_path(synset1)
    path2 = hypernyms_path(synset2)

    min_dist = len(path1) + len(path2) + 2
    hypernyms = []

    for i in range(len(path1)):
        for j in range(len(path2)):
            if path1[i] == path2[j]:
                if i + j < min_dist:
                    min_dist = i + j
                    hypernyms = path1[i]
                    break

                if i + j == min_dist:
                    hypernyms += path1[i]
                    break

    return hypernyms

synset1 = get_synset('tree')
synset2 = get_synset('air')
print(closest_hypernyms(synset1, synset2))

Synset('physical_entity.n.01')


## 6. Word similarity

In [8]:
def sort_similarity(synset, synset_list):
    def synset_similarity(synset_comp):
        return synset.path_similarity(synset_comp)

    synset_list.sort(key=synset_similarity, reverse=True)
    return synset_list

synset = get_synset('cat')
synset_list = get_synset_list(['animal', 'tree', 'house', 'object', 'public_school', 'mouse'])
print(sort_similarity(synset, synset_list))

[Synset('mouse.n.01'), Synset('animal.n.01'), Synset('object.n.01'), Synset('tree.n.01'), Synset('house.n.01'), Synset('public_school.n.01')]


## 7. Indirect meronyms

In [10]:
def indirect_meronyms(synset1, synset2):
    def holonyms_path(synset):
        holonyms = []
        while True:
            holonyms += [synset]
            synset = holonyms_meronyms(synset1)[0]

            if not synset or synset[0] == holonyms[-1]:
                break
            synset = synset[0]
        return holonyms
    
    holonyms1 = holonyms_path(synset1)
    holonyms2 = holonyms_path(synset2)

    for holonym in holonyms1:
        if holonym in holonyms2:
            return True

    for holonym in holonyms2:
        if holonym in holonyms1:
            return True

    return False

word1 = 'tree'
word2 = 'air'
print(f'Are "{word1}" and "{word2}" indirect meronyms?', indirect_meronyms(get_synset(word1), get_synset(word2)))

Are "tree" and "air" indirect meronyms? True


## 8. Synonyms and antonyms

In [11]:
def synonyms_antonyms(word):
    def get_adjectives(synset_list):
        return [synset for synset in synset_list if synset.pos() == 'a']

    def get_names(synset_list):
        return [synset.lemmas()[0].name() for synset in synset_list]
    
    def get_lemma_names(synset_list):
        return [synset.name() for synset in synset_list]

    synset_list = get_adjectives(wordnet.synsets(word))

    for synset in synset_list:
        get_gloss(synset)
        print('Synonyms: ', get_names(synset.similar_tos()))
        print('Antonyms: ', get_lemma_names(synset.lemmas()[0].antonyms()))

synonyms_antonyms('strong')

strong.a.01: having strength or power greater than average or expected
Synonyms:  ['beardown', 'beefed-up', 'brawny', 'bullnecked', 'bullocky', 'fortified', 'hard', 'industrial-strength', 'ironlike', 'knock-down', 'noticeable', 'reinforced', 'robust', 'stiff', 'vehement', 'virile', 'well-knit']
Antonyms:  ['weak']
potent.a.03: having a strong physiological or chemical effect; ; ; ,
Synonyms:  ['equipotent', 'multipotent']
Antonyms:  ['impotent']
